In [ ]:
#-------------------------------------------------------------------------------------JUPYTER NOTEBOOK SETTINGS-------------------------------------------------------------------------------------
from IPython.core.display import display, HTML                                    
display(HTML("<style>.container { width:100% !important; }</style>"))  

In [ ]:
import os
import gc
import re
import librosa
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from joblib import dump, load

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping 

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
model = load_model('saved_data/models/non-masked_mobilenetv3-finetuned/mobilenetv3small_finetuned_model.keras')
history = load('saved_data/models/non-masked_custom_cnn/custom-cnn_training_history.joblib')
x_test, y_test = load('saved_data/test_data.joblib')
x_test = np.array(x_test, dtype=np.float32)

all_labels = ['battery', 'description', 'environment', 'greeting', 'health', 'noise', 'nutrition', 'silence', 'sun', 'water']
label_encoder = LabelEncoder()
label_encoder.fit(all_labels)

# Predictions with the model
y_pred = np.argmax(model.predict(x_test), axis=1)

# Decode the integer predictions back to string labels
y_pred_labels = label_encoder.inverse_transform(y_pred)

# Encode y_test if it's not already in integer form
if isinstance(y_test[0], str):
    y_test_encoded = label_encoder.transform(y_test)
else:
    y_test_encoded = y_test

# Compute confusion matrix with integer labels for calculation
conf_matrix = confusion_matrix(y_test_encoded, y_pred)

# Classification Report with string labels
print("Number of epochs trained:", len(history['loss']))
print(classification_report(y_test_encoded, y_pred, target_names=label_encoder.classes_))

In [ ]:
# Plot confusion matrix with string labels for readability
fig, ax = plt.subplots(figsize=(8, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Greens', ax=ax, 
            xticklabels=label_encoder.classes_, 
            yticklabels=label_encoder.classes_)
ax.set_xlabel('Predicted Labels')
ax.set_ylabel('True Labels')
ax.set_title('Confusion Matrix')
plt.show()

# Loss and Accuracy Per Epoch plots
# plt.figure(figsize=(12, 6))
# plt.subplot(1, 2, 1)
# plt.plot(history['loss'], label='Training Loss')
# plt.plot(history['val_loss'], label='Validation Loss')
# plt.title('Loss Per Epoch')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend()

# plt.subplot(1, 2, 2)
# plt.plot(history['accuracy'], label='Training Accuracy')
# plt.plot(history['val_accuracy'], label='Validation Accuracy')
# plt.title('Accuracy Per Epoch')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.legend()
# plt.tight_layout()
# plt.show()